In [1]:
import pandas as pd

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter

from pathlib import Path

import glob

In [2]:
def priceparse(s):
    s = s.replace('Ft','')
    s = ''.join(s.split())
    try:
        return int(s)
    except:
        return pd.np.nan
    
def teljesitmeny(x):
    try:
        return int(x.replace(" LE",""))
    except:
        try:    
            return int(x.replace(" LE,",""))
        except:
            return pd.np.nan
def evjarat(x):
    try:
        return 2019 - int(str(x)[:4])
    except:
        return pd.np.nan
    
def MEGTETT(x):
    try: 
        return  int(x.replace("km","").replace("\xa0",""))
    except:
        return pd.np.nan

In [3]:
def heggesztes(data):    
    x=pd.read_csv(data)

    x['price-parsed'] = x['price'].apply(priceparse)
    x['MARKA'] = x['link'].apply(lambda x: x.split('/')[4])
    x['UZEMANYAG'] = [f.strip().replace(",","") for f in x["adat-0"]]
    x["TELJESITMENY LOERO"] = x["adat-4"].apply(teljesitmeny)

    x['MARKA'] = [f.strip() for f in x['MARKA']]
    x['MARKA_encoded'] = LabelEncoder().fit_transform(x['MARKA'])
    #x[['MARKA', 'MARKA_encoded']] # special syntax to get just these two columns
    marka = x['MARKA_encoded']
    max(marka)

    pd.value_counts(x['MARKA_encoded'].values, sort=False)
    x = x.rename(index=str, columns={"CD-RĂDIĂ“": "CD-RADIO"})
    x = x.rename(index=str, columns={"BĹ\x90R": "BLR"})
    x = x.rename(index=str, columns={"ELSĹ\x90 TULAJDONOSTĂ“L": "ELSO_TULAJTOL"})
    x = x.rename(index=str, columns={"FĹ°THETĹ\x90 ĂśLĂ‰S": "FUTHETO_ULES"})
    x = x.rename(index=str, columns={"GARĂ\x81ZSBAN TARTOTT": "GARAZSBAN_TARTOTT"})
    x = x.rename(index=str, columns={"MĂ\x81SODIK TULAJDONOSTĂ“L": "MASODIK_TULAJ"})
    x = x.rename(index=str, columns={"TETĹ\x90MONITOR": "TETO_MONITOR"})
    x = x.rename(index=str, columns={"GARANCIĂ\x81LIS": "GARANCIALIS"})
    x = x.rename(index=str, columns={"CENTRĂ\x81LZĂ\x81R": "CENTRALZAR"})

    varnames = list(x.columns.values)
    hajtaslanc = x['UZEMANYAG'].values.tolist()
    hajtaslanc = list(Counter(hajtaslanc).keys())

    x['UZEMANYAG_encoded'] = LabelEncoder().fit_transform(x['UZEMANYAG'])
    uzemanyag = x['UZEMANYAG']


    x['freq'] = x.groupby('MARKA_encoded')['MARKA_encoded'].transform('count')


    x["hajtaslanc_freq"] = x.groupby('UZEMANYAG')['UZEMANYAG'].transform('count')
    
    x = x.join(pd.get_dummies(x["UZEMANYAG"])[["Benzin","Dízel","Elektromos","Hibrid","Hibrid (Benzin)","Hibrid (Dízel)"]])
    x = x.join(pd.get_dummies(x["MARKA"]))

    x["hany_eves"] = x["adat-1"].apply(evjarat)
    x["hany_eves_2"] = x["hany_eves"]**2
    x["MEGTETT"] = x["adat-5"].apply(MEGTETT)
    x["MEGTETT_2"] = x["MEGTETT"]**2
    
    x["datum"] = data[-14:-4]


    x.to_csv(data[14:-4]+"_cleaned"+".csv")

In [8]:
data_folder = Path("../1_scraping/data/*.csv")

for data in glob.glob(str(data_folder)):
    print(data)

..\1_scraping\data\hasznaltauto-2019-02-12.csv
..\1_scraping\data\hasznaltauto-2019-03-12.csv
..\1_scraping\data\hasznaltauto-2019-05-18.csv


In [9]:
for data in glob.glob(str(data_folder)):
    heggesztes(data)

In [ ]:
"""x=pd.read_csv(glob.glob("../1_scraping/data/*csv")[0])

x['price-parsed'] = x['price'].apply(priceparse)
x['MARKA'] = x['link'].apply(lambda x: x.split('/')[4])
x['UZEMANYAG'] = [f.strip().replace(",","") for f in x["adat-0"]]
x["TELJESITMENY LOERO"] = x["adat-4"].apply(teljesitmeny)

x['MARKA'] = [f.strip() for f in x['MARKA']]
x['MARKA_encoded'] = LabelEncoder().fit_transform(x['MARKA'])
#x[['MARKA', 'MARKA_encoded']] # special syntax to get just these two columns
marka = x['MARKA_encoded']
max(marka)

pd.value_counts(x['MARKA_encoded'].values, sort=False)
x = x.rename(index=str, columns={"CD-RĂDIĂ“": "CD-RADIO"})
x = x.rename(index=str, columns={"BĹ\x90R": "BLR"})
x = x.rename(index=str, columns={"ELSĹ\x90 TULAJDONOSTĂ“L": "ELSO_TULAJTOL"})
x = x.rename(index=str, columns={"FĹ°THETĹ\x90 ĂśLĂ‰S": "FUTHETO_ULES"})
x = x.rename(index=str, columns={"GARĂ\x81ZSBAN TARTOTT": "GARAZSBAN_TARTOTT"})
x = x.rename(index=str, columns={"MĂ\x81SODIK TULAJDONOSTĂ“L": "MASODIK_TULAJ"})
x = x.rename(index=str, columns={"TETĹ\x90MONITOR": "TETO_MONITOR"})
x = x.rename(index=str, columns={"GARANCIĂ\x81LIS": "GARANCIALIS"})
x = x.rename(index=str, columns={"CENTRĂ\x81LZĂ\x81R": "CENTRALZAR"})

varnames = list(x.columns.values)
hajtaslanc = x['UZEMANYAG'].values.tolist()
hajtaslanc = list(Counter(hajtaslanc).keys())

x['UZEMANYAG_encoded'] = LabelEncoder().fit_transform(x['UZEMANYAG'])
uzemanyag = x['UZEMANYAG']


x['freq'] = x.groupby('MARKA_encoded')['MARKA_encoded'].transform('count')


x["hajtaslanc_freq"] = x.groupby('UZEMANYAG')['UZEMANYAG'].transform('count')

x = x.join(pd.get_dummies(x["UZEMANYAG"])[["Benzin","Dízel","Elektromos","Hibrid","Hibrid (Benzin)","Hibrid (Dízel)"]])
x = x.join(pd.get_dummies(x["MARKA"]))

x["hany_eves"] = x["adat-1"].apply(evjarat)
x["hany_eves_2"] = x["hany_eves"]**2
x["MEGTETT"] = x["adat-5"].apply(MEGTETT)
x["MEGTETT_2"] = x["MEGTETT"]**2


x.head()"""